In [21]:
import pandas as pd
import networkx as nx
import numpy as np
import requests
import geopandas as gpd
import osmnx as ox

In [85]:
df_grid = pd.read_excel('./grid.xlsx', index_col=0)
df_substations = pd.read_excel('./substations.xlsx', index_col=0)

pd.set_option('display.max_rows', 100)

In [86]:
df_tielines = pd.read_excel('./jao_static_grid.xlsx', sheet_name='Tielines', skiprows=[0])
df_tielines = df_tielines[[
    'NE_name', 'EIC_Code', 'TSO', 
    'Full_name', 'Full_name.1', 'Voltage_level(kV)', 
    'Resistance_R(Ω)', 'Reactance_X(Ω)', 'Susceptance_B(μS)', 'Length_(km)'
]]
df_tielines = df_tielines.rename(columns={
    'NE_name': 'name', 
    'EIC_Code': 'eic',
    'TSO': 'tso',
    'Voltage_level(kV)': 'voltage',
    'Resistance_R(Ω)': 'resistance',
    'Reactance_X(Ω)': 'reactance',
    'Susceptance_B(μS)': 'susceptance',
    'Length_(km)': 'length',
    'Full_name': 'substation_1',
    'Full_name.1': 'substation_2',
})

tso_zone_map = {
    '50HERTZ': 'DE', 
    'Amprion GmbH': 'DE', 
    'APG': 'AT', 
    'CEPS': 'CZ', 
    'Creos': 'DE', 
    'ELES': 'SI', 
    'ELIA': 'BE',
    'HOPS': 'HR', 
    'MAVIR': 'HU', 
    'PSE': 'PL', 
    'RTE': 'FR', 
    'SEPS': 'SK', 
    'TEL': 'RO', 
    'TENNET NL': 'NL',
    'TENNETGMBH': 'DE', 
    'TRANSNETBW': 'DE'
}

df_tielines['zone'] = df_tielines.apply(lambda row : tso_zone_map[row.tso], axis=1)

In [87]:
def get_from_zone(row):
    if row['eic'][3] == '-':
        return row['eic'][4] + row['eic'][5]
    else:
        return row['zone']
    
def get_to_zone(row):
    if row['eic'][3] == '-':
        return row['eic'][7] + row['eic'][8]
    else:
        return row['zone']

df_tielines['from_zone'] = df_tielines.apply(lambda row: get_from_zone(row), axis=1)
df_tielines['to_zone'] = df_tielines.apply(lambda row: get_to_zone(row), axis=1)

z1 = set(df_tielines['from_zone'].unique())
z1 = z1.union(df_tielines['to_zone'].unique())
zones = list(z1)

relevant_zones = ['DE', 'AT', 'CZ', 'DE', 'SI', 'BE', 'HR', 'HU', 'PL', 'FR', 'SK', 'RO', 'NL', 'D2', 'D4', 'D7', 'D8']

df_tielines = df_tielines[(df_tielines['from_zone'].isin(relevant_zones)) & (df_tielines['to_zone'].isin(relevant_zones))]

In [88]:
substations = df_substations.name.tolist()
substations = list(map(str.strip,substations))
tieline_substations = []

for index, row in df_tielines.iterrows():
    if row['substation_1'] not in tieline_substations:
        tieline_substations.append(row['substation_1'])
    if row['substation_2'] not in tieline_substations:
        tieline_substations.append(row['substation_2'])
        
new_substations = set(list(map(str.lower,tieline_substations))).difference(list(map(str.lower,substations)))

df_substations_extra = pd.DataFrame(list(zip(new_substations.copy())), columns =['name'])
df_substations_extra['zone'] = None
df_substations_extra['tso'] = None
df_substations_extra['lat'] = None
df_substations_extra['lon'] = None

In [89]:
exclude_list = ['mukachevo', 'zahidno ukrainska']
df_substations_extra = df_substations_extra[~df_substations_extra.name.isin(exclude_list)]
display(df_substations_extra)
df_substations_extra.to_excel("tieline_substations.xlsx")

,name,zone,tso,lat,lon
0,buers,None,None,None,None
1,liskovec,None,None,None,None
2,repiquage bifferdange 220kv,None,None,None,None
3,y-laufenburg,None,None,None,None
4,mont-saint-martin,None,None,None,None
5,vulcanesti,None,None,None,None
6,vill,None,None,None,None
7,sotel,None,None,None,None
8,monceau,None,None,None,None
9,mt-saint-martin,None,None,None,None


In [ ]:
ox.settings.timeout = 3

for index, row in df_substations_extra.iterrows():
    try:
        tags = {
            'power_source': True,
            'substation': True,
            'electricity': True,
        }
        osm_find = ox.geometries_from_address(row['name'], tags)
        if len(osm_find) > 0:
            print(osm_find.iloc[0])
            if hasattr(osm_find, "name"):
                df_substations_extra.loc[index, 'osm_find_name'] = osm_find.name[0]
            if hasattr(osm_find, "power"):
                df_substations_extra.loc[index, 'osm_find_type'] = osm_find.power[0]
            elif hasattr(osm_find, "substation"):
                df_substations_extra.loc[index, 'osm_find_type'] = osm_find.substation[0]
            if hasattr(osm_find, "operator"):
                df_substations_extra.loc[index, 'osm_find_operator'] = osm_find.operator[0]
            df_substations_extra.loc[index, 'lat'] = osm_find.geometry[0].representative_point().y
            df_substations_extra.loc[index, 'lon'] = osm_find.geometry[0].representative_point().x
        else:
            print('Not found: ', row['name'], ', ', index)
    except ValueError as ve:
        print('Value error, ', row['name'], ', ', index)
    except requests.exceptions.RequestException as rt:
        print('Timeout, ', row['name'], ', ', index)

Not found:  buers  ,  0
Not found:  liskovec  ,  1
Value error,  repiquage bifferdange 220kv ,  2
Not found:  y-laufenburg ,  3
Not found:  mont-saint-martin ,  4
Not found:  vulcanesti ,  5
Timeout,  vill ,  6
